# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 19 2023 10:39AM,261405,19,N18036,"NAPP Technologies, LLC",Released
1,Apr 19 2023 10:39AM,261405,19,N18023,"NAPP Technologies, LLC",Released
2,Apr 19 2023 10:26AM,261403,10,9395392,Emerginnova,Released
3,Apr 19 2023 10:21AM,261399,19,2022103591,"GUSA Granules USA, Inc.",Released
4,Apr 19 2023 10:20AM,261398,19,ADV80016050,"AdvaGen Pharma, Ltd",Released
5,Apr 19 2023 10:20AM,261398,19,ADV80016051,"AdvaGen Pharma, Ltd",Released
6,Apr 19 2023 10:20AM,261398,19,ADV80016052,"AdvaGen Pharma, Ltd",Released
7,Apr 19 2023 10:20AM,261398,19,ADV80016053,"AdvaGen Pharma, Ltd",Released
8,Apr 19 2023 10:20AM,261398,19,ADV80016054,"AdvaGen Pharma, Ltd",Released
9,Apr 19 2023 10:20AM,261398,19,ADV80016055,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,261397,Released,3
23,261398,Released,42
24,261399,Released,1
25,261403,Released,1
26,261405,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261397,NaN,3.0
261398,NaN,42.0
261399,NaN,1.0
261403,NaN,1.0
261405,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
260869,3.0,30.0
260910,8.0,5.0
261117,9.0,1.0
261321,0.0,25.0
261331,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
260869,3,30
260910,8,5
261117,9,1
261321,0,25
261331,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,260869,3,30
1,260910,8,5
2,261117,9,1
3,261321,0,25
4,261331,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,260869,3,30
1,260910,8,5
2,261117,9,1
3,261321,,25
4,261331,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 19 2023 10:39AM,261405,19,"NAPP Technologies, LLC"
2,Apr 19 2023 10:26AM,261403,10,Emerginnova
3,Apr 19 2023 10:21AM,261399,19,"GUSA Granules USA, Inc."
4,Apr 19 2023 10:20AM,261398,19,"AdvaGen Pharma, Ltd"
46,Apr 19 2023 10:07AM,261397,10,"CLINUVEL, Inc."
49,Apr 19 2023 10:02AM,261396,10,Eywa Pharma Inc.
55,Apr 19 2023 9:58AM,261395,10,"Methapharm, Inc."
60,Apr 19 2023 9:45AM,261321,15,"Person & Covey, Inc."
85,Apr 19 2023 9:45AM,261362,15,"Alliance Pharma, Inc."
86,Apr 19 2023 9:41AM,261366,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 19 2023 10:39AM,261405,19,"NAPP Technologies, LLC",,2
1,Apr 19 2023 10:26AM,261403,10,Emerginnova,,1
2,Apr 19 2023 10:21AM,261399,19,"GUSA Granules USA, Inc.",,1
3,Apr 19 2023 10:20AM,261398,19,"AdvaGen Pharma, Ltd",,42
4,Apr 19 2023 10:07AM,261397,10,"CLINUVEL, Inc.",,3
5,Apr 19 2023 10:02AM,261396,10,Eywa Pharma Inc.,5,1
6,Apr 19 2023 9:58AM,261395,10,"Methapharm, Inc.",,5
7,Apr 19 2023 9:45AM,261321,15,"Person & Covey, Inc.",,25
8,Apr 19 2023 9:45AM,261362,15,"Alliance Pharma, Inc.",,1
9,Apr 19 2023 9:41AM,261366,15,"Alliance Pharma, Inc.",7,21


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 10:39AM,261405,19,"NAPP Technologies, LLC",2,
1,Apr 19 2023 10:26AM,261403,10,Emerginnova,1,
2,Apr 19 2023 10:21AM,261399,19,"GUSA Granules USA, Inc.",1,
3,Apr 19 2023 10:20AM,261398,19,"AdvaGen Pharma, Ltd",42,
4,Apr 19 2023 10:07AM,261397,10,"CLINUVEL, Inc.",3,
5,Apr 19 2023 10:02AM,261396,10,Eywa Pharma Inc.,1,5
6,Apr 19 2023 9:58AM,261395,10,"Methapharm, Inc.",5,
7,Apr 19 2023 9:45AM,261321,15,"Person & Covey, Inc.",25,
8,Apr 19 2023 9:45AM,261362,15,"Alliance Pharma, Inc.",1,
9,Apr 19 2023 9:41AM,261366,15,"Alliance Pharma, Inc.",21,7


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 10:39AM,261405,19,"NAPP Technologies, LLC",2,
1,Apr 19 2023 10:26AM,261403,10,Emerginnova,1,
2,Apr 19 2023 10:21AM,261399,19,"GUSA Granules USA, Inc.",1,
3,Apr 19 2023 10:20AM,261398,19,"AdvaGen Pharma, Ltd",42,
4,Apr 19 2023 10:07AM,261397,10,"CLINUVEL, Inc.",3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 10:39AM,261405,19,"NAPP Technologies, LLC",2.0,NaN
1,Apr 19 2023 10:26AM,261403,10,Emerginnova,1.0,NaN
2,Apr 19 2023 10:21AM,261399,19,"GUSA Granules USA, Inc.",1.0,NaN
3,Apr 19 2023 10:20AM,261398,19,"AdvaGen Pharma, Ltd",42.0,NaN
4,Apr 19 2023 10:07AM,261397,10,"CLINUVEL, Inc.",3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 10:39AM,261405,19,"NAPP Technologies, LLC",2.0,0.0
1,Apr 19 2023 10:26AM,261403,10,Emerginnova,1.0,0.0
2,Apr 19 2023 10:21AM,261399,19,"GUSA Granules USA, Inc.",1.0,0.0
3,Apr 19 2023 10:20AM,261398,19,"AdvaGen Pharma, Ltd",42.0,0.0
4,Apr 19 2023 10:07AM,261397,10,"CLINUVEL, Inc.",3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2352434,22.0,6.0
15,784049,47.0,7.0
16,261388,1.0,0.0
19,1568257,74.0,0.0
20,782896,36.0,20.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2352434,22.0,6.0
1,15,784049,47.0,7.0
2,16,261388,1.0,0.0
3,19,1568257,74.0,0.0
4,20,782896,36.0,20.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,22.0,6.0
1,15,47.0,7.0
2,16,1.0,0.0
3,19,74.0,0.0
4,20,36.0,20.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,22.0
1,15,Released,47.0
2,16,Released,1.0
3,19,Released,74.0
4,20,Released,36.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Executing,6.0,7.0,0.0,0.0,20.0
Released,22.0,47.0,1.0,74.0,36.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Executing,6.0,7.0,0.0,0.0,20.0
1,Released,22.0,47.0,1.0,74.0,36.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Executing,6.0,7.0,0.0,0.0,20.0
1,Released,22.0,47.0,1.0,74.0,36.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()